In [66]:
# CartPole Env
import gym
import matplotlib as plt

class CartPoleEnv:
    def __init__(self):
        self.fps = 30
        self.env = gym.make('CartPole-v0')  # environment interface to game
        self.env.seed(42)
        self.env.reset()
        self.is_show = False
        self.current_sate = []
        self.is_done = False

    def render(self):
        self.env.render()
        
    def reset(self, is_show = False):
        state = self.env.reset()
        state = state.tolist()
        self.current_sate = state
        self.is_done = False
        self.is_show = is_show
        if self.is_show:
            self.env.render()

        return state
    
    def act(self, action):
        if self.is_show:
            self.env.render()
        # return observation, reward, done, info
        state, reward, is_done, info = self.env.step(self.get_action_set()[action])
        state = state.tolist()
        self.current_sate = state
        self.is_done = is_done
        return state, reward, is_done, info

    def get_num_actions(self):
        return 2

    def get_action_set(self):
        return [0, 1]

    def get_screen_rgb(self):
        pass

    def get_screen_gray(self):
        pass

    def get_num_state_features(self):
        return 4

    def get_state(self):
        return self.current_sate

    def is_over(self):
        return self.is_done


In [67]:
# !pip install pygame
# !git clone https://github.com/ntasfi/PyGame-Learning-Environment.git
# !pip install -e ./PyGame-Learning-Environment

In [68]:
from ple.games.flappybird import FlappyBird as flappyBird
from ple import PLE

class FlappyBirdEnv:
    def __init__(self):
        self.fps = 30
        self.game = flappyBird()
        self.env = PLE(self.game, fps=self.fps, display_screen=False)  # environment interface to game
        self.env.reset_game()

    def reset(self, is_show = False):
        self.env = PLE(self.game, fps=self.fps, display_screen=is_show)  # environment interface to game
        self.env.reset_game()
        state = self.get_state()

        return state
    
    def act(self, action):
        # return state_prime, reward, done, info
        reward = self.env.act(self.env.getActionSet()[action])
#         print(reward)
        
        # Survive reward +1
#         reward += 1
        
        # Get closer to the middle of top and bottom pipe and get more reward
#         state = self.game.getGameState()
        
#         next_dis_to_mid = abs((state['next_pipe_top_y'] + state['next_pipe_bottom_y']) / 2 - state['player_y'])
#         print('State')
#         print(state)
#         print('Mid')
#         print((state['next_pipe_top_y'] + state['next_pipe_bottom_y']) / 2)
#         print('next_dis_to_mid')
#         print(abs((state['next_pipe_top_y'] + state['next_pipe_bottom_y']) / 2 - state['player_y']))
#         next_next_dis_to_mid = abs((state['next_next_pipe_top_y'] + state['next_next_pipe_bottom_y']) / 2 - state['player_y'])
#         dis_reward_coef = 0.01
        
#         reward += dis_reward_coef * ((-next_dis_to_mid) + 0.5 * (-next_next_dis_to_mid))
        
        state_prime = self.get_state()
        is_done = self.is_over()
        info = ""
        return state_prime, reward, is_done, info

    def get_num_actions(self):
        return len(self.env.getActionSet())

    def get_action_set(self):
        return self.env.getActionSet()

    def get_screen_rgb(self):
        return self.env.getScreenRGB()

    def get_screen_gray(self):
        return self.env.getScreenGrayscale()

    def get_num_state_features(self):
        return len(self.game.getGameState())

    def get_state(self):
        # dict
        #     * player y position.
        #     * players velocity.
        #     * next pipe distance to player
        #     * next pipe top y position
        #     * next pipe bottom y position
        #     * next next pipe distance to player
        #     * next next pipe top y position
        #     * next next pipe bottom y position

        # state = {
        #     "player_y": self.player.pos_y,
        #     "player_vel": self.player.vel,
            
        #     "next_pipe_dist_to_player": next_pipe.x - self.player.pos_x,
        #     "next_pipe_top_y": next_pipe.gap_start,
        #     "next_pipe_bottom_y": next_pipe.gap_start+self.pipe_gap, 
            
        #     "next_next_pipe_dist_to_player": next_next_pipe.x - self.player.pos_x,
        #     "next_next_pipe_top_y": next_next_pipe.gap_start,
        #     "next_next_pipe_bottom_y": next_next_pipe.gap_start+self.pipe_gap 
        # }
        state = self.game.getGameState()
        state['next_pipe_top_y'] -= state['player_y']
        state['next_pipe_bottom_y'] -= state['player_y']
        state['next_next_pipe_top_y'] -= state['player_y']
        state['next_next_pipe_bottom_y'] -= state['player_y']
        return list(state.values())

    def is_over(self):
        return self.env.game_over()


In [69]:
import numpy as np

class EpsilonGreedy:
    def __init__(self, epsilon, num_action, min_epsilon = 0.01, decay = 0.99):
        self.epsilon = epsilon
        self.num_action = num_action
        self.min_epsilon = min_epsilon
        self.decay = decay
        self.action_time = 0

    def select_action(self):
        self.action_time += 1
        if np.random.rand() < self.epsilon:
            self.update_epsilon()
            return np.random.choice(self.num_action)
        else: 
            self.update_epsilon()
            return -1
    
    def update_epsilon(self):
        self.epsilon = max(self.min_epsilon, min(0.5, 0.99**(self.action_time / 30)))
    
    def shutdown_explore(self):
        self.epsilon = 0

In [70]:
# ddqn.py
# Implement DDQN with Delay Network, History Replay, Epsilon Greedy
# 2020/08/10 SYC

import tensorflow as tf
import numpy as np
# import matplotlib as plt
# import policiesPractice.models.expStrategy as stg

class Agent:
    def __init__(self, state_size, num_action, delay_update_every_iter, reward_discount, learning_rate, exploration_strategy):
        self.state_size = state_size
        self.num_action = num_action
        self.reward_discount = reward_discount
        self.exploration_strategy = exploration_strategy
        self.delay_update_every_iter = delay_update_every_iter
        self.iter = 0
        self.eps = 0
        self.data_type = tf.float32
        self.batch_size = 32
        self.optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
        self.avg_loss = tf.keras.metrics.Mean(name = 'loss')
        self.online_model = self.build_model('online')
        self.target_model = self.build_model('target')
        self.is_shutdown_explore = False

        self.buffer = []
        self.buffer_size = 50000
    
    def build_model(self, name):
        nn_input = tf.keras.Input(shape = self.state_size, dtype = self.data_type)

        x = tf.keras.layers.Dense(units = 128)(nn_input)
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.Dense(units = 128)(x)
        x = tf.keras.layers.ReLU()(x)
        nn_output = tf.keras.layers.Dense(units = self.num_action)(x)

        model = tf.keras.Model(name = name, inputs = nn_input, outputs = nn_output)

        return model

    def predict(self, state):
        # Online Model
        return self.online_model(tf.convert_to_tensor(state, self.data_type))

    def max_q(self, state_primes):
        # Target Model
        return tf.reduce_max(self.target_model(tf.convert_to_tensor(state_primes, self.data_type)), axis = 1)

    def loss(self, states, actions, rewards, state_primes):
        predicts = self.predict(states)
        indice = tf.stack([tf.range(len(actions)), actions], axis = 1)
        predict_qs = tf.gather_nd(predicts, indice)

        target_qs = self.max_q(state_primes)
        # Compute loss as formular: loss = E((r + gamma * max(Q(s', a'| Theta')) - Q(s, a, | Theta))^2)
        # Update model with a batch Every time. As a result, we compute the Expectation(E) of the total loss of a batch.
        return tf.reduce_mean(tf.square(rewards + self.reward_discount * target_qs - predict_qs))

    def get_metrics_loss(self):
        return self.avg_loss.result()
    
    def reset_metrics_loss(self):
        self.avg_loss.reset_states()

    def select_action(self, state):
        # Assume using Epsilon Greedy Strategy
        action = self.exploration_strategy.select_action()
        act_qs = self.predict([state])
        # If the index of action (return value) is -1, choose the action with highest probability that online_model predict
        if action == -1:
            return tf.argmax(act_qs, axis = 1)[0], act_qs
        else:
            # If the index of action (return value) is != -1, act randomly    
            return action, act_qs

    def shutdown_explore(self):
        self.is_shutdown_explore = True

    def update(self, loss, tape):
        gradients = tape.gradient(loss, self.online_model.trainable_variables)
        # gradients = [gradients if gradients is not None else tf.zeros_like(var) for var, grad in zip(self.model.trainable_variables, gradients)]
        self.optimizer.apply_gradients(zip(gradients, self.online_model.trainable_variables))
        self.avg_loss.update_state(loss)

        # Update exploration rate of Epsilon Greedy Strategy
        self.exploration_strategy.update_epsilon()
        
        # Deplay Update
        is_update_target = False
        if self.iter % self.delay_update_every_iter == 0:
            self.target_model.set_weights(self.online_model.get_weights())
            is_update_target = True

        return is_update_target
    
#     def preprocess_state(self, env_state):
#         # Preprocess SINGLE state
#         return list(env_state.values())

#     def preprocess_states(self, env_states):
#         # Preprocess MULTIPLE states
#         state_list = []
#         for env_state in env_states:
#             state_list.append(list(env_state.values()))

#         return state_list

    def add_buffer(self, new_state, new_action, new_reward, new_state_prime):
        # Add ONE action-state pair every time
        self.buffer.append({'state': new_state, 'action': new_action, 'reward': new_reward, 'state_prime': new_state_prime})
        if len(self.buffer) > self.buffer_size:
            self.buffer.pop(0)
    
    def sample(self, num_sample):
        if num_sample >= len(self.buffer):
            idx_samples = np.random.choice(len(self.buffer), num_sample).tolist()
        else:
            idx_samples = np.random.choice(len(self.buffer), num_sample, replace = False).tolist()

        states = []
        actions = []
        rewards = []
        state_primes = []
#         act_qs = []

        for idx in idx_samples:
            states.append(self.buffer[idx]['state'])
            actions.append(self.buffer[idx]['action'])
            rewards.append(self.buffer[idx]['reward'])
            state_primes.append(self.buffer[idx]['state_prime'])
#             act_qs.append(self.buffer[idx]['act_qs'])

        return states, actions, rewards, state_primes #, act_qs
    
    def train_on_env(self, env, is_show = False):
        episode_reward = 0
        episode_loss = 0
        count = 0
        state = env.reset(is_show)

        while not env.is_over():
            action, act_qs = self.select_action(state)
            state_prime, reward, is_done, info = env.act(action)

            # print(f'B State: {state}, Action: {action}, Reward: {reward}, State_Prime: {state_prime}')

            self.add_buffer(state, action, reward, state_prime)
            sample_states, sample_actions, sample_rewards, sample_state_primes = self.sample(self.batch_size)
            #  Update model with TD
            with tf.GradientTape() as tape:
                tape.watch(self.online_model.trainable_variables)
                loss = self.loss(sample_states, sample_actions, sample_rewards, sample_state_primes)
                is_update_target = self.update(loss, tape)

                self.iter += 1
                count += 1

                state = state_prime
                episode_reward += reward
                episode_loss += loss
                
        env.reset()
        self.eps += 1

        return episode_reward, (episode_loss / count)

In [ ]:
# FlappyBird-DDQN Experiment
# 2020/08/11 SYC 

# import models.A2C as A2C
# import models.expStrategy.epsilonGreedy as EPSG
# import envs.cartPole as cartPole
import models.util as Util
import logging
import matplotlib.pyplot as plt
from matplotlib.pylab import figure
import os
import numpy as np
# To run tqdm on notebook, import tqdm.notebook
from tqdm.notebook import tqdm
# Run on pure python
# from tqdm import tqdm

# Config Logging format
# logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
# Config logging module to enable on notebook
# logger = logging.getLogger()
# logger.setLevel(logging.DEBUG)

# Block any pop-up windows
os.environ['SDL_VIDEODRIVER'] = 'dummy'

# Test GPU and show the available logical & physical GPUs
Util.test_gpu()

env = FlappyBirdEnv()
# env = CartPoleEnv()
NUM_STATE_FEATURES = env.get_num_state_features()
NUM_ACTIONS = env.get_num_actions()
EPISODE_NUM = 10000
PRINT_EVERY_EPISODE = 50
LEARNING_RATE = 0.003
REWARD_DISCOUNT = 0.99
DELAY_EVERY_ITER = 1000

exp_stg = EpsilonGreedy(0.2, NUM_ACTIONS)
agent = Agent((NUM_STATE_FEATURES, ), NUM_ACTIONS, DELAY_EVERY_ITER, REWARD_DISCOUNT, LEARNING_RATE, exp_stg)

state = env.reset()
accum_reward = 0

# tqdm progress bar
bar = []
# Reward & LossHistory
r_his = []
avg_r_his = [0]
loss_his = []
episode_reward = 0

print("Episode 1")
for episode in range(1, EPISODE_NUM + 1):
    if episode % PRINT_EVERY_EPISODE == 1:
        if episode > 1:
            bar.close()
            print("Avgerage Accumulated Reward: {} | Loss: {}".format(round(accum_reward / PRINT_EVERY_EPISODE), agent.get_metrics_loss()))
            print("Episode {}".format(episode))
            agent.reset_metrics_loss()
            avg_r_his.append(round(accum_reward / PRINT_EVERY_EPISODE))
            accum_reward = 0
        bar = tqdm(total = PRINT_EVERY_EPISODE)

    episode_reward, episode_loss = agent.train_on_env(env)
    accum_reward += episode_reward
    r_his.append(episode_reward)
    loss_his.append(episode_loss)
    
    episode_reward = 0

    bar.update(1)        
    env.reset()

bar.close()    
print("Accumulated Reward: {} | Loss: {}".format(round(accum_reward / PRINT_EVERY_EPISODE), agent.get_metrics_loss()))
avg_r_his.append(round(accum_reward / PRINT_EVERY_EPISODE))
agent.reset_metrics_loss()

# Evaluate the model
agent.shutdown_explore()
agent.reset_metrics_loss()
# Reset Game
env_state = env.reset()
accum_reward = 0

while not env.is_over():
    # env.render()
    action, act_log_prob = agent.select_action(state)
    state_prime, reward, is_done, info = env.act(action)

    state = state_prime
    accum_reward += reward

print("Evaluate")
print("Accumulated Reward: {}".format(round(accum_reward)))

# Plot Reward History
# figure(num=None, figsize=(24, 6), dpi=80)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(24, 6), dpi=80)
fig.suptitle(f'FlappyBird DDQN Result (Evaluate Reward: {round(accum_reward)})')
x_datas = range(0, len(r_his))
avg_x_datas = range(0, EPISODE_NUM + 1, PRINT_EVERY_EPISODE)

ax1.plot(x_datas, r_his, color='blue')
ax1.plot(avg_x_datas, avg_r_his, color='red')
ax1.set_xlabel('Episodes')
ax1.set_ylabel('Reward / Episode')
ax1.grid()

ax2.plot(x_datas, loss_his, color='orange')
ax2.set_xlabel('Episodes')
ax2.set_ylabel('Loss / Episode')
ax2.grid()

plt.savefig('flappyBird-DDQN-res.svg')
plt.show()

4 Physical GPUs, 1 Logical GPUs
Episode 1



Avgerage Accumulated Reward: -5 | Loss: 13.07143783569336
Episode 51



Avgerage Accumulated Reward: -5 | Loss: 1.8814830780029297
Episode 101



Avgerage Accumulated Reward: -5 | Loss: 2.406684637069702
Episode 151



Avgerage Accumulated Reward: -5 | Loss: 4.795338153839111
Episode 201


...


Avgerage Accumulated Reward: -4 | Loss: 0.73581463098526
Episode 6201



Avgerage Accumulated Reward: -5 | Loss: 1.031221628189087
Episode 6251



Avgerage Accumulated Reward: -5 | Loss: 1.0788319110870361
Episode 6301



Avgerage Accumulated Reward: -5 | Loss: 1.553877830505371
Episode 6351



Avgerage Accumulated Reward: -5 | Loss: 1.5797085762023926
Episode 6401
